In [125]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler

In [126]:
#Conditions
# MIN NO OF SKILLS FOR AN ENTRY: 5
# MAX NO OF SKILLS FOR AN ENTRY: 10

#Variables
skill_count = 9

In [127]:
dict1 = [{'ID':1,'country':'Sri Lanka','skill_set':['math','science','music','guitar']},
        {'ID':2,'country':'England','skill_set':['singing','presentation']},
        {'ID':3,'country':'USA','skill_set':['music','guitar','piano']},
        {'ID':4,'country':'Sri Lanka','skill_set':['presentation','chemistry','physics']},
        {'ID':5,'country':'Japan','skill_set':['math','music','guitar','piano']}
]

In [128]:
dict2 = [{'ID':1,'country':'Sri Lanka','skill_set':['math','guitar'],'rec_count':5},
        {'ID':2,'country':'England','skill_set':['singing','presentation'],'rec_count':7},
        {'ID':3,'country':'USA','skill_set':['guitar','piano'],'rec_count':2},
        {'ID':4,'country':'Sri Lanka','skill_set':['presentation','physics'],'rec_count':1},
        {'ID':5,'country':'USA','skill_set':['math','piano'],'rec_count':8},
        {'ID':6,'country':'India','skill_set':['math','piano','singing','guitar'],'rec_count':0}
]

In [129]:
def preprocess(dict):
  df = pd.DataFrame(dict)

  mlb = MultiLabelBinarizer()
  df_v2 = pd.DataFrame(mlb.fit_transform(df['skill_set']),columns=mlb.classes_)
  df_merge_col = pd.merge(df, df_v2, left_index=True, right_index=True)
  dataset = df_merge_col.drop(['skill_set'],axis=1)

  return dataset

In [130]:
def find_tutors(student,tutors_df,country=False,n=5):
  '''
  input: pandas series of a specific student entry
  country: whether to consider the country or not
  returns: similar tutors
  '''
  if country:
    tutors_df_mod = tutors_df[tutors_df['country']==student['country']]

  student = student.drop(['country','ID'])
  tutors_df_mod = tutors_df.drop(['country','ID','rec_count'],axis=1)

  mask = sum([tutors_df_mod==student])
  print(mask)
  sim_scores = mask.sum(axis=1)

  sim_tuts = sim_scores.nlargest(n)
  sim = tutors_df[tutors_df.index.isin(sim_tuts.index)]
  sim['sim_score'] = sim_tuts

  return sim

In [131]:
def rec_score(sim_tuts_df):
    '''
  input: similar tutors dataframe
  w_sim: similarity weight
  w_rec: reccomender count weight
  returns: modified dataframe with final recommending score
  '''
  sim_tuts_df_copy = sim_tuts_df.copy()
  sim_tuts_df_copy['sim_score'] = MinMaxScaler().fit_transform(np.array(sim_tuts_df['sim_score']).reshape(-1,1))
  # sim_tuts_df_copy['rec_count'] = MinMaxScaler().fit_transform(np.array(sim_tuts_df['rec_count']).reshape(-1,1))

  tot_rec_count = sum(sim_tuts_df_copy['rec_count'])
  print(tot_rec_count)
  sim_tuts_df_copy['rec_score'] = (tot_rec_count / ( sim_tuts_df_copy['rec_count'] +1 ))
  sim_tuts_df_copy['rec_score'] = MinMaxScaler().fit_transform(np.array(sim_tuts_df_copy['rec_score']).reshape(-1,1))
  sim_tuts_df_copy['final_score'] = sim_tuts_df_copy['sim_score'] + sim_tuts_df_copy['rec_score']

  return sim_tuts_df_copy

In [132]:
students_df = preprocess(dict1)

In [133]:
tutors_df = preprocess(dict2)
student = students_df.iloc[2]
tutors_df[tutors_df['country']==student['country']]


,ID,country,rec_count,guitar,math,physics,piano,presentation,singing
2,3,USA,2,1,0,0,1,0,0
4,5,USA,8,0,1,0,1,0,0


In [134]:
sim_tuts_df = find_tutors(student,tutors_df)
sim_tuts_df

   chemistry  guitar  math  music  ...  piano  presentation  science  singing
0          0       1     0      0  ...      0             1        0        1
1          0       0     1      0  ...      0             0        0        0
2          0       1     1      0  ...      1             1        0        1
3          0       0     1      0  ...      0             0        0        1
4          0       0     0      0  ...      1             1        0        1
5          0       1     0      0  ...      1             1        0        0

[6 rows x 9 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,country,rec_count,guitar,math,physics,piano,presentation,singing,sim_score
0,1,Sri Lanka,5,1,1,0,0,0,0,4
1,2,England,7,0,0,0,0,1,1,2
2,3,USA,2,1,0,0,1,0,0,6
4,5,USA,8,0,1,0,1,0,0,4
5,6,India,0,1,1,0,1,0,1,4


In [135]:
sim_tuts_df_scores = rec_score(sim_tuts_df)
sim_tuts_df_scores

22


,ID,country,rec_count,guitar,math,physics,piano,presentation,singing,sim_score,rec_score,final_score
0,1,Sri Lanka,5,1,1,0,0,0,0,0.5,0.062500,0.562500
1,2,England,7,0,0,0,0,1,1,0.0,0.015625,0.015625
2,3,USA,2,1,0,0,1,0,0,1.0,0.250000,1.250000
4,5,USA,8,0,1,0,1,0,0,0.5,0.000000,0.500000
5,6,India,0,1,1,0,1,0,1,0.5,1.000000,1.500000
